In [1]:
pip install transformers

     |████████████████████████████████| 1.5MB 10.9MB/s 
     |████████████████████████████████| 2.9MB 45.2MB/s 
     |████████████████████████████████| 890kB 61.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f5e91a18c65286472ae8c9b10d5298dfbbd1a2ed17963ff19b8cf8cc90ffd240
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
pip install sentencepiece

     |████████████████████████████████| 1.1MB 8.8MB/s 


Entailnment


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

max_length = 256

premise = "The Clinton followers kept to the higher ground in the discussion."
hypothesis = "The Clinton surrogates also held the high ground in the context war."

hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
#hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
#hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name)
model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)


tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                     max_length=max_length,
                                                     return_token_type_ids=True, truncation=True)
input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
    # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

outputs = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None)
    # Note:
    # "id2label": {
    #     "0": "entailment",
    #     "1": "neutral",
    #     "2": "contradiction"
    # },

predicted_probability = torch.softmax(outputs[0], dim=1)[0].tolist()  # batch_size only one

print("Premise:", premise)
print("Hypothesis:", hypothesis)
print("Entailment:", predicted_probability[0])
print("Neutral:", predicted_probability[1])
print("Contradiction:", predicted_probability[2])
    

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Premise: The Clinton followers kept to the higher ground in the discussion.
Hypothesis: The Clinton surrogates also held the high ground in the context war.
Entailment: 0.13277667760849
Neutral: 0.8394918441772461
Contradiction: 0.027731474488973618


Coreferencing

In [2]:

!git clone https://github.com/huggingface/neuralcoref.git
!pip install -U spacy
!python -m spacy download en

%cd neuralcoref

!pip install -r requirements.txt
!pip install -e .


import neuralcoref
import spacy

nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

doc1 = nlp('My sister has a dog. She loves him.')
print(doc1._.coref_clusters)

[Errno 2] No such file or directory: 'temp'
/content
Cloning into 'neuralcoref'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 758 (delta 2), reused 3 (delta 1), pack-reused 748
Receiving objects: 100% (758/758), 67.83 MiB | 33.58 MiB/s, done.
Resolving deltas: 100% (399/399), done.
     |████████████████████████████████| 10.4MB 4.1MB/s 
     |████████████████████████████████| 1.1MB 38.1MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 12.1MB 2.6MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-cp36-none-any.whl size=12047109 sha256=bbf0004f3b0fd9e70c3ebd83d922e3b7f3b4d76d47a9ad59e6aefb7d3b601a3d
  Stored in directory: /tmp/pip-ephem-wheel-

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)
100%|██████████| 40155833/40155833 [00:00<00:00, 70742489.69B/s]


[My sister: [My sister, She], a dog: [a dog, him]]


Semantic Equivalence

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")

model = AutoModelForMaskedLM.from_pretrained("bert-base-cased-finetuned-mrpc")

Some weights of the model checkpoint at bert-base-cased-finetuned-mrpc were not used when initializing BertForMaskedLM: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-cased-finetuned-mrpc and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
You

In [25]:
premise = "The Clinton followers kept to the higher ground in the discussion."
hypothesis = "The Clinton surrogates also held the high ground in the context war."

tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                     max_length=max_length,
                                                     return_token_type_ids=True, truncation=True)
input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)

token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

outputs = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None)

predicted_probability = torch.softmax(outputs[0], dim=1)[0][0].tolist()  # batch_size only one

print("Premise:", premise)
print("Hypothesis:", hypothesis)
#print("Entailment:", predicted_probability[0])
#print("Neutral:", predicted_probability[1])
#print("Contradiction:", predicted_probability[1])
print(predicted_probability[0])
print(outputs[0])


Premise: The Clinton followers kept to the higher ground in the discussion.
Hypothesis: The Clinton surrogates also held the high ground in the context war.
0.029433228075504303
